In [9]:
import torch
import torch.nn as nn
import torchvision
from torchvision.ops import sigmoid_focal_loss

In [12]:
output = torch.rand(2, 30)
target = torch.randint(0, 2, (2,30), dtype=torch.float32)

In [17]:
loss = sigmoid_focal_loss(output, target, reduction='mean')

tensor(0.1404)

In [3]:
class QA_with_head(nn.Module):
    def __init__(self, config, device):
        super(QA_with_head, self).__init__()

        self.config = config 
        self.model = AutoModelForQuestionAnswering.from_pretrained(self.config.model.model_path)
        if config.model.two_step_loss:
            self.score=nn.Linear(config.model.dim,1)
            self.loss_fct=nn.BCEWithLogitsLoss()
        self.classifier_hidden = torch.nn.Linear(128, 128)
        self.classifier_dropout = torch.nn.Dropout(p=0.2)
        self.output_layer = torch.nn.Linear(128, 1)
        self.sigmoid = torch.nn.Sigmoid()
        self.loss_classifier = torch.nn.BCELoss()
        self.device = device

    def forward(self, batch):
        if not self.config.model.non_pooler:
            out = self.model(input_ids = batch["question_context_input_ids"].to(self.device), 
                            attention_mask = batch["question_context_attention_mask"].to(self.device),
                            token_type_ids = batch["question_context_token_type_ids"].to(self.device),
                            start_positions = batch["start_positions"].to(self.device),
                            end_positions = batch["end_positions"].to(self.device),
                            output_hidden_states=True)
        else:
            out = self.model(input_ids = batch["question_context_input_ids"].to(self.device), 
                            attention_mask = batch["question_context_attention_mask"].to(self.device),
                            start_positions = batch["start_positions"].to(self.device),
                            end_positions = batch["end_positions"].to(self.device),
                            output_hidden_states=True)
        if self.config.model.two_step_loss:
            cls_tokens=out.hidden_states[-1][:,0]
            scores=self.score(cls_tokens) # [32,1]
            out.loss+=self.loss_fct(scores,batch["answerable"])

            return (out,torch.nn.functional.softmax(scores))

        if out.loss != None :
            cls_representations = out["hidden_states"][-1][:, 0, :]
            cls_representations = self.classifier_hidden(cls_representations)
            cls_representations = self.classifier_dropout(cls_representations)
            cls_representations = self.output_layer(cls_representations)
            cls_representations = cls_representations.squeeze(dim=-1)
            cls_representations = self.sigmoid(cls_representations)
            answerable = torch.tensor(batch["answerable"],dtype = torch.float32)
            clf_loss = self.loss_classifier(cls_representations, answerable)
            out.loss += clf_loss

        return out  

    def export_to_onnx(self, tokenizer):
        # TODO Using torch.onnx.export
        # Will use transformers.onnx.export for transformer models

        # TODO Using transformers.onnx if this doesn't work
        feature = "question-answering"

        # load config
        model_kind, model_onnx_config = FeaturesManager.check_supported_model_or_raise(self.model, feature=feature)
        onnx_config = model_onnx_config(self.model.config)

        # export
        onnx_inputs, onnx_outputs = transformers.onnx.export(
                preprocessor=tokenizer,
                model=self.model,
                config=onnx_config,
                opset=13,
                output=Path("checkpoints/{}/model.onnx".format(self.config.load_path))
        )

        print(onnx_inputs, onnx_outputs)

In [4]:
question = "How many programming languages does BLOOM support?"
context = "BLOOM has 176 billion parameters and can generate text in 46 languages natural languages and 13 programming languages."

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/bert_uncased_L-2_H-128_A-2")

Downloading: 100%|██████████| 382/382 [00:00<00:00, 164kB/s]
Downloading: 100%|██████████| 232k/232k [00:03<00:00, 70.6kB/s] 


In [6]:
inputs = tokenizer([question, question], [context, context], return_tensors="pt")

In [7]:
inputs

{'input_ids': tensor([[  101,  2129,  2116,  4730,  4155,  2515, 13426,  2490,  1029,   102,
         13426,  2038, 18561,  4551, 11709,  1998,  2064,  9699,  3793,  1999,
          4805,  4155,  3019,  4155,  1998,  2410,  4730,  4155,  1012,   102],
        [  101,  2129,  2116,  4730,  4155,  2515, 13426,  2490,  1029,   102,
         13426,  2038, 18561,  4551, 11709,  1998,  2064,  9699,  3793,  1999,
          4805,  4155,  3019,  4155,  1998,  2410,  4730,  4155,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1]])}